In [4]:
import numpy as np
rng = np.random.default_rng()

In [83]:
def joint_multifractal(p,q, n):
    P = p.copy()
    Q = q.copy()
    for i in range(n):
        P = np.kron(P, p)
        Q = np.kron(Q, q)
    return(np.stack([P,Q], axis=-1))



def joint_logmultifractal(n, sigma1=1, sigma2=1, corr=0):
    
    PQ = np.exp(rng.multivariate_normal(np.array([0,0]),  np.array([[sigma1**2,corr*sigma1*sigma2], [corr*sigma1*sigma2, sigma2**2]]), 4))
    
    P = PQ[:,0].reshape(2,2)
    Q = PQ[:,1].reshape(2,2)
    for i in range(n):
        PQ = np.exp(rng.multivariate_normal(np.array([0,0]),np.array([[sigma1**2,corr*sigma1*sigma2], [corr*sigma1*sigma2, sigma2**2]]), P.shape[0]**2 * 4))
        P = np.kron(P, np.ones((2,2))) 
        P = P * PQ[:,0].reshape(P.shape)
        Q = np.kron(Q, np.ones((2,2))) 
        Q = Q * PQ[:,1].reshape(P.shape)
    return(np.stack([P,Q], axis=-1))
    

In [61]:
cascade_simple = joint_multifractal(np.array([[0.25,0.25],[0.1,0.4]]),
                   np.array([[0.1,0.1],[0.7,0.1]]),
                   8)

In [96]:
test = np.stack([joint_logmultifractal(8,corr=l, sigma1=0.5, sigma2=0.5 ) for l in np.arange(-0.9,1.1,0.1)])

In [97]:
np.savez_compressed('lognormalcascade.npz', test)

In [64]:
cascade_lognormal = joint_logmultifractal(8, 0.5)

In [65]:
np.savez('cascade_simple_log.npz',  cascade_simple, cascade_lognormal)